In [182]:
import json
import urllib
from urllib import request, parse, error
import requests
import base64
import numpy
from scipy.stats import chisquare
from scipy.stats import fisher_exact

In [213]:
def Query_Panther(genes):
    url = "https://www.pantherdb.org/services/oai/pantherdb/enrich/overrep?geneInputList=" \
        + genes \
        + "&organism=9606&annotDataSet=GO%3A0003674&enrichmentTestType=FISHER&correction=NONE"
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    data = json.loads(response.read())
    return data
#0003674 molecular function
#0008150 biological process
#0005575 cellular component

In [214]:
def Retrieve_Panther_Results(inputGenes):
    goDict = {}
    genes = ""
    geneCounter = 0
    mappedGenes = 0
    for line in inputGenes:
        genes = genes + "," + line.rstrip()
        geneCounter += 1
    
        if geneCounter > 4000:
            data = Query_Panther(genes)
            for entry in data["results"]["result"]:
                if entry["term"]["label"] not in goDict:
                    goDict[entry["term"]["label"]] = 0
                goDict[entry["term"]["label"]] += entry["number_in_list"]
            mappedGenes += data["results"]["input_list"]["mapped_count"]
            genes = ""
            geneCounter = 0
            
    data = Query_Panther(genes)
    mappedGenes += data["results"]["input_list"]["mapped_count"]
    for entry in data["results"]["result"]:
        if entry["term"]["label"] not in goDict:
            goDict[entry["term"]["label"]] = 0
        goDict[entry["term"]["label"]] += entry["number_in_list"]   
    return goDict, mappedGenes


In [259]:
cell = "MCF-7"

In [260]:
dataDir = "/home/moorej3/Lab/ENCODE/Encyclopedia/V7/Registry/V7-hg38/3D-Links/PLS-comparisons/"
input1 = open(dataDir+"Panther-Common-" + cell + ".txt")
goDict1, count1 = Retrieve_Panther_Results(input1)
input1.close()

input2 = open(dataDir+"Panther-RNAPII-" + cell + ".txt")
goDict2, count2 = Retrieve_Panther_Results(input2)
input2.close()

input3 = open(dataDir+"Panther-CTCF-" + cell + ".txt")
goDict3, count3 = Retrieve_Panther_Results(input3)
input3.close()

input4 = open(dataDir+"Panther-None-" + cell + ".txt")
goDict4, count4 = Retrieve_Panther_Results(input4)
input4.close()

input5 = open(dataDir+"Panther-Background-" + cell + ".txt")
goDict5, count5 = Retrieve_Panther_Results(input5)
input5.close()

In [254]:
term = "transmembrane signaling receptor activity"
fraction = goDict5[term]/count5
observed = [goDict1[term], goDict2[term], goDict3[term], goDict4[term]]
expected = [count1 * fraction, count2 * fraction, count3 * fraction, count4 * fraction] 
p = chisquare(observed, expected)[1]
ratio = numpy.divide(numpy.array(observed),numpy.array(expected))
termNew = term.replace(" ","_")
padj = p * len(goDict1)
print(termNew, "\t", '{:.2e}'.format(padj), "\t", round(ratio[0],2), "\t", round(ratio[1],2), "\t", round(ratio[2],2), "\t", round(ratio[3],2))

transmembrane_signaling_receptor_activity 	 5.38e-08 	 1.13 	 0.6 	 2.94 	 1.72


In [261]:
term = "nucleic acid binding"
fraction = goDict5[term]/count5
observed = [goDict1[term], goDict2[term], goDict3[term], goDict4[term]]
expected = [count1 * fraction, count2 * fraction, count3 * fraction, count4 * fraction] 
p = chisquare(observed, expected)[1]
ratio = numpy.divide(numpy.array(observed),numpy.array(expected))
termNew = term.replace(" ","_")
padj = p * len(goDict1)
print(termNew, "\t", '{:.2e}'.format(padj), "\t", round(ratio[0],2), "\t", round(ratio[1],2), "\t", round(ratio[2],2), "\t", round(ratio[3],2))

nucleic_acid_binding 	 9.98e-15 	 0.95 	 1.19 	 0.59 	 0.88


In [262]:
#outputFile=open("test-MCF.txt", "w+")
for term in goDict5:
    fraction = goDict5[term]/count5
    observed = [goDict1[term], goDict2[term], goDict3[term], goDict4[term]]
    expected = [count1 * fraction, count2 * fraction, count3 * fraction, count4 * fraction] 
    p = chisquare(observed, expected)[1]
    ratio = numpy.divide(numpy.array(observed),numpy.array(expected))
    termNew = term.replace(" ","_")
    if p < correctedP and goDict5[term] >= 25:
        print(termNew, "\t", '{:.2e}'.format(p), "\t", round(ratio[0],2), "\t", round(ratio[1],2), "\t", round(ratio[2],2), "\t", round(ratio[3],2))
#outputFile.close()

organic_cyclic_compound_binding 	 1.07e-09 	 0.98 	 1.11 	 0.74 	 0.93
heterocyclic_compound_binding 	 1.09e-10 	 0.99 	 1.11 	 0.71 	 0.92
G_protein-coupled_receptor_activity 	 6.79e-06 	 0.72 	 0.7 	 1.85 	 1.52
RNA_binding 	 5.79e-19 	 1.01 	 1.26 	 0.48 	 0.72
nucleic_acid_binding 	 1.97e-18 	 0.95 	 1.19 	 0.59 	 0.88
transmembrane_signaling_receptor_activity 	 2.76e-12 	 0.75 	 0.68 	 1.85 	 1.47
molecular_transducer_activity 	 8.94e-13 	 0.82 	 0.72 	 1.9 	 1.37
signaling_receptor_activity 	 8.94e-13 	 0.82 	 0.72 	 1.9 	 1.37
catalytic_activity,_acting_on_a_nucleic_acid 	 5.17e-06 	 1.14 	 1.09 	 0.18 	 0.84
cadherin_binding 	 1.80e-06 	 1.13 	 1.35 	 0.62 	 0.61
signaling_receptor_regulator_activity 	 1.03e-11 	 1.34 	 0.54 	 2.78 	 1.09
signaling_receptor_activator_activity 	 1.38e-11 	 1.33 	 0.53 	 2.85 	 1.04
receptor_ligand_activity 	 3.30e-12 	 1.35 	 0.51 	 2.94 	 1.01
DNA_binding 	 2.34e-07 	 0.89 	 1.15 	 0.67 	 0.99
structural_constituent_of_ribosome 	 7.31e-07 	 0.8

<ipython-input-262-d5acc506d369>:7: RuntimeWarning: invalid value encountered in true_divide
  ratio = numpy.divide(numpy.array(observed),numpy.array(expected))


 	 1.24e-06 	 1.15 	 0.79 	 1.37 	 1.16
ligand-gated_channel_activity 	 8.40e-09 	 0.22 	 0.67 	 3.95 	 1.46
voltage-gated_potassium_channel_activity 	 1.34e-21 	 0.5 	 0.32 	 6.71 	 1.29
voltage-gated_monoatomic_ion_channel_activity 	 1.25e-18 	 0.46 	 0.44 	 4.34 	 1.56
voltage-gated_channel_activity 	 3.33e-18 	 0.5 	 0.43 	 4.29 	 1.55
potassium_channel_regulator_activity 	 7.23e-06 	 0.83 	 1.17 	 4.97 	 0.72
potassium_channel_activity 	 2.28e-19 	 0.47 	 0.4 	 5.8 	 1.35
ligand-gated_monoatomic_cation_channel_activity 	 1.55e-08 	 0.17 	 0.68 	 4.29 	 1.35
inorganic_cation_transmembrane_transporter_activity 	 4.86e-11 	 0.88 	 0.69 	 2.46 	 1.17
growth_factor_activity 	 6.94e-06 	 1.68 	 0.52 	 2.46 	 0.86
salt_transmembrane_transporter_activity 	 3.48e-09 	 0.86 	 0.69 	 2.25 	 1.24
transmembrane_transporter_activity 	 1.05e-07 	 0.96 	 0.79 	 1.8 	 1.11
monoatomic_ion_transmembrane_transporter_activity 	 2.26e-09 	 0.86 	 0.75 	 2.15 	 1.18
metal_ion_transmembrane_transporter_a

In [211]:
for term in goDict5:
    observed = [goDict1[term], count1-goDict1[term]]
    expected = [goDict2[term], count2-goDict2[term]]
    p = fisher_exact([observed,expected])[1]
    ratio = numpy.divide(numpy.array(observed),numpy.array(expected))
    termNew = term.replace(" ","_")
    if p < correctedP and goDict5[term] >= 25:
        print(termNew, "\t", '{:.2e}'.format(p), "\t", round(ratio[0],2), "\t", round(goDict1[term]/count1,2), "\t", round(goDict2[term]/count2,2))

<ipython-input-211-3572270caaee>:5: RuntimeWarning: invalid value encountered in true_divide
  ratio = numpy.divide(numpy.array(observed),numpy.array(expected))


regulation_of_nucleobase-containing_compound_metabolic_process 	 3.02e-07 	 0.53 	 0.23 	 0.28
regulation_of_gene_expression 	 8.84e-07 	 0.54 	 0.26 	 0.31
regulation_of_RNA_metabolic_process 	 5.02e-07 	 0.52 	 0.21 	 0.26
regulation_of_DNA-templated_transcription 	 1.38e-06 	 0.52 	 0.19 	 0.23
regulation_of_RNA_biosynthetic_process 	 2.22e-06 	 0.53 	 0.19 	 0.23


<ipython-input-211-3572270caaee>:5: RuntimeWarning: divide by zero encountered in true_divide
  ratio = numpy.divide(numpy.array(observed),numpy.array(expected))


In [267]:
for term in goDict5:
    observed = [goDict3[term], count3-goDict3[term]]
    expected = [goDict5[term], count5-goDict5[term]]
    p = fisher_exact([observed,expected])[1]
    termNew = term.replace(" ","_")
    if p < correctedP and goDict5[term] >= 25:
        print(termNew, "\t", '{:.2e}'.format(p), "\t", round(goDict3[term]/count3*100,2), "\t", round(goDict5[term]/count5*100,2))

organic_cyclic_compound_binding 	 1.87e-07 	 27.93 	 37.84
heterocyclic_compound_binding 	 8.09e-09 	 26.58 	 37.47
RNA_binding 	 3.51e-07 	 5.56 	 11.54
nucleic_acid_binding 	 1.25e-10 	 15.02 	 25.62
catalytic_activity,_acting_on_a_nucleic_acid 	 2.45e-07 	 0.75 	 4.25
signaling_receptor_regulator_activity 	 3.57e-06 	 4.35 	 1.57
catalytic_activity,_acting_on_RNA 	 9.89e-06 	 0.3 	 2.68
signaling_receptor_activator_activity 	 3.51e-06 	 4.2 	 1.48
receptor_ligand_activity 	 2.08e-06 	 4.2 	 1.43
voltage-gated_monoatomic_cation_channel_activity 	 1.77e-07 	 2.85 	 0.59
monoatomic_ion_gated_channel_activity 	 3.62e-08 	 4.35 	 1.22
gated_channel_activity 	 4.76e-08 	 4.35 	 1.23
channel_activity 	 5.27e-08 	 5.71 	 1.99
passive_transmembrane_transporter_activity 	 5.85e-08 	 5.71 	 2.0
monoatomic_ion_channel_activity 	 1.49e-06 	 4.8 	 1.75
monoatomic_cation_channel_activity 	 7.60e-07 	 4.2 	 1.35
voltage-gated_potassium_channel_activity 	 6.01e-09 	 2.7 	 0.4
voltage-gated_monoatomi

In [269]:
dataDir = "/home/moorej3/Lab/ENCODE/Encyclopedia/V7/Registry/V7-hg38/Cell-Type-Specific/"
input1 = open(dataDir+"low-low.genes")
goDict1, count1 = Retrieve_Panther_Results(input1)
input1.close()

input2 = open(dataDir+"low-high.genes")
goDict2, count2 = Retrieve_Panther_Results(input2)
input2.close()


In [272]:
for term in goDict1:
    observed = [goDict2[term], count2-goDict2[term]]
    expected = [goDict1[term], count1-goDict1[term]]
    p = fisher_exact([observed,expected])[1]
    termNew = term.replace(" ","_")
    padj = p * len(goDict1)
    if goDict5[term] >= 25:
        print(termNew, "\t", '{:.2e}'.format(padj), "\t", round(goDict1[term]/count2*100,2), "\t", round(goDict1[term]/count1*100,2))

RNA_binding 	 2.82e+03 	 4.4 	 2.69
cytoskeletal_protein_binding 	 2.84e+03 	 16.69 	 10.2
UNCLASSIFIED 	 1.20e+03 	 10.02 	 6.12
molecular_function 	 1.20e+03 	 153.57 	 93.88
binding 	 2.13e+03 	 142.94 	 87.38
protein_binding 	 2.82e+03 	 126.86 	 77.55
actin_binding 	 4.12e+03 	 7.74 	 4.73
catalytic_activity,_acting_on_a_nucleic_acid 	 5.06e+03 	 1.37 	 0.83
ion_binding 	 1.65e+03 	 59.18 	 36.18
kinesin_binding 	 4.81e+02 	 1.82 	 1.11
tubulin_binding 	 3.53e+03 	 6.37 	 3.9
gated_channel_activity 	 3.97e+03 	 5.77 	 3.53
RNA_polymerase_II_cis-regulatory_region_sequence-specific_DNA_binding 	 8.15e+01 	 14.72 	 9.0
monoatomic_ion_gated_channel_activity 	 4.51e+03 	 5.61 	 3.43
cis-regulatory_region_sequence-specific_DNA_binding 	 6.48e+01 	 14.87 	 9.09
monoatomic_cation_channel_activity 	 1.99e+03 	 5.61 	 3.43
channel_activity 	 1.36e+03 	 7.59 	 4.64
passive_transmembrane_transporter_activity 	 1.36e+03 	 7.59 	 4.64
DNA-binding_transcription_factor_activity 	 1.58e+02 	 16.69

ubiquitin_protein_ligase_binding 	 2.13e+03 	 3.49 	 2.13
transferase_activity,_transferring_one-carbon_groups 	 5.06e+03 	 0.91 	 0.56
inorganic_anion_transmembrane_transporter_activity 	 1.60e+03 	 1.97 	 1.21
cytokine_activity 	 1.96e+03 	 0.91 	 0.56
syntaxin_binding 	 3.91e+03 	 1.06 	 0.65
cell_adhesion_molecule_binding 	 4.52e+03 	 5.77 	 3.53
ubiquitin-like_protein_ligase_binding 	 2.64e+03 	 3.64 	 2.23
histone_modifying_activity 	 2.11e+03 	 0.76 	 0.46
cadherin_binding 	 3.07e+03 	 3.64 	 2.23
phosphatidylinositol-3-phosphate_binding 	 8.27e+02 	 0.76 	 0.46
endopeptidase_inhibitor_activity 	 3.32e+03 	 0.61 	 0.37
general_transcription_initiation_factor_activity 	 5.06e+03 	 0.0 	 0.0
3'-5'_exonuclease_activity 	 5.06e+03 	 0.0 	 0.0
lipid_transfer_activity 	 1.04e+02 	 0.0 	 0.0
calmodulin_binding 	 3.82e+02 	 2.43 	 1.48
oxidoreductase_activity,_acting_on_NAD(P)H,_quinone_or_similar_compound_as_acceptor 	 5.06e+03 	 0.0 	 0.0
polyubiquitin_modification-dependent_protein_b

transferase_activity 	 5.06e+03 	 19.42 	 11.87
ubiquitin_binding 	 3.44e+03 	 0.46 	 0.28
N-methyltransferase_activity 	 1.48e+03 	 0.46 	 0.28
nucleobase-containing_compound_kinase_activity 	 5.06e+03 	 0.46 	 0.28
active_transmembrane_transporter_activity 	 4.32e+03 	 2.88 	 1.76
G_protein_activity 	 5.06e+03 	 0.46 	 0.28
cysteine-type_endopeptidase_regulator_activity_involved_in_apoptotic_process 	 5.06e+03 	 0.46 	 0.28
intramembrane_lipid_transporter_activity 	 5.06e+03 	 0.46 	 0.28
signal_sequence_binding 	 2.84e+03 	 0.15 	 0.09
oxidoreductase_activity,_acting_on_the_aldehyde_or_oxo_group_of_donors 	 7.89e+02 	 0.15 	 0.09
cis-trans_isomerase_activity 	 5.06e+03 	 0.15 	 0.09
cyclin-dependent_protein_serine/threonine_kinase_regulator_activity 	 7.89e+02 	 0.15 	 0.09
carbohydrate_derivative_transmembrane_transporter_activity 	 5.06e+03 	 0.15 	 0.09
metalloendopeptidase_activity 	 3.91e+03 	 1.06 	 0.65
nucleobase-containing_compound_transmembrane_transporter_activity 	 5.06e

hydrolase_activity,_acting_on_glycosyl_bonds 	 3.91e+03 	 1.06 	 0.65
histone_H3_methyltransferase_activity 	 2.67e+03 	 0.3 	 0.19
isomerase_activity 	 8.39e+02 	 1.21 	 0.74
DNA_secondary_structure_binding 	 5.06e+03 	 0.15 	 0.09
deacetylase_activity 	 5.06e+03 	 0.15 	 0.09
peptidase_regulator_activity 	 3.12e+03 	 1.82 	 1.11
transcription_coactivator_activity 	 6.53e+02 	 2.12 	 1.3
enzyme_inhibitor_activity 	 1.69e+03 	 3.19 	 1.95
chromatin_binding 	 2.72e+03 	 4.7 	 2.88
